In [1]:
## Use this codes to lookup the diagnosis codes, lab proc, medication
## Author: Ahmed Abdelhameed
## Date: July 06, 2022

import pandas as pd
import numpy as np
import csv
from collections import OrderedDict
import re
#import path as path

# Load code descriptions

In [2]:
# import the whole lookup table to check merging
import pandas as pd
raw = pd.read_csv('/ctao_shared_data/Optum_lookup/lu_diagnosis.csv', delimiter='\t', low_memory=False)
raw

,diag_cd,diag_desc,diag_fst3_cd,diag_fst3_desc,diag_fst4_cd,diag_fst4_desc,gdr_spec_cd,mdc_cd_desc,mdc_code,icd_ver_cd
0,23297,UNKNOWN DIAGNOSIS,232,UNKNOWN DIAGNOSIS,2329,UNKNOWN DIAGNOSIS,NaN,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,9
1,84110,UNKNOWN DIAGNOSIS,841,UNKNOWN DIAGNOSIS,8411,UNKNOWN DIAGNOSIS,NaN,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,9
2,23663,UNKNOWN DIAGNOSIS,236,UNKNOWN DIAGNOSIS,2366,UNKNOWN DIAGNOSIS,NaN,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,9
3,91120,UNKNOWN DIAGNOSIS,911,UNKNOWN DIAGNOSIS,9112,UNKNOWN DIAGNOSIS,NaN,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,9
4,CP590,UNKNOWN DIAGNOSIS,CP5,UNKNOWN DIAGNOSIS,CP59,UNKNOWN DIAGNOSIS,NaN,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,9
...,...,...,...,...,...,...,...,...,...,...
316774,S72433C,DSPL FX MED C UNS FEM INIT OPEN III,S72,FRACTURE OF FEMUR,S724,FRACTURE OF LOWER END OF FEMUR,NaN,DISEASES & DISORDERS OF THE MUSCULOSKELETAL SYSTE,08,10
316775,I0741,UNKNOWN DIAGNOSIS,I07,UNKNOWN DIAGNOSIS,I074,UNKNOWN DIAGNOSIS,NaN,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,10
316776,K13159,UNKNOWN DIAGNOSIS,K13,UNKNOWN DIAGNOSIS,K131,UNKNOWN DIAGNOSIS,NaN,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,10
316777,V80730,ANIMAL-RIDER INJ COLL STREETCAR,V80,ANML-RIDR/OCC ANML VEH INJ TRNS ACC,V807,ANML-RIDR/OCC ANML VEH INJ COLL NMV,NaN,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,10


In [3]:
#raw[raw["diag_cd"] == "L851"]
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316779 entries, 0 to 316778
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   diag_cd         316777 non-null  object
 1   diag_desc       316779 non-null  object
 2   diag_fst3_cd    316696 non-null  object
 3   diag_fst3_desc  316779 non-null  object
 4   diag_fst4_cd    306578 non-null  object
 5   diag_fst4_desc  306411 non-null  object
 6   gdr_spec_cd     6108 non-null    object
 7   mdc_cd_desc     316779 non-null  object
 8   mdc_code        316779 non-null  object
 9   icd_ver_cd      316779 non-null  object
dtypes: object(10)
memory usage: 24.2+ MB


## Filter Some columns

In [4]:
lu = raw[['diag_cd','diag_desc','diag_fst3_cd','diag_fst3_desc', 'diag_fst4_cd','diag_fst4_desc','icd_ver_cd']]
#lu = lu.loc[lu.diag_desc!='UNKNOWN DIAGNOSIS']
lu = lu.drop_duplicates().reset_index(drop=True)
lu.isnull().sum()

diag_cd               2
diag_desc             0
diag_fst3_cd         83
diag_fst3_desc        0
diag_fst4_cd      10201
diag_fst4_desc    10368
icd_ver_cd            0
dtype: int64

In [5]:
lu

,diag_cd,diag_desc,diag_fst3_cd,diag_fst3_desc,diag_fst4_cd,diag_fst4_desc,icd_ver_cd
0,23297,UNKNOWN DIAGNOSIS,232,UNKNOWN DIAGNOSIS,2329,UNKNOWN DIAGNOSIS,9
1,84110,UNKNOWN DIAGNOSIS,841,UNKNOWN DIAGNOSIS,8411,UNKNOWN DIAGNOSIS,9
2,23663,UNKNOWN DIAGNOSIS,236,UNKNOWN DIAGNOSIS,2366,UNKNOWN DIAGNOSIS,9
3,91120,UNKNOWN DIAGNOSIS,911,UNKNOWN DIAGNOSIS,9112,UNKNOWN DIAGNOSIS,9
4,CP590,UNKNOWN DIAGNOSIS,CP5,UNKNOWN DIAGNOSIS,CP59,UNKNOWN DIAGNOSIS,9
...,...,...,...,...,...,...,...
316774,S72433C,DSPL FX MED C UNS FEM INIT OPEN III,S72,FRACTURE OF FEMUR,S724,FRACTURE OF LOWER END OF FEMUR,10
316775,I0741,UNKNOWN DIAGNOSIS,I07,UNKNOWN DIAGNOSIS,I074,UNKNOWN DIAGNOSIS,10
316776,K13159,UNKNOWN DIAGNOSIS,K13,UNKNOWN DIAGNOSIS,K131,UNKNOWN DIAGNOSIS,10
316777,V80730,ANIMAL-RIDER INJ COLL STREETCAR,V80,ANML-RIDR/OCC ANML VEH INJ TRNS ACC,V807,ANML-RIDR/OCC ANML VEH INJ COLL NMV,10


In [6]:
lu[lu["diag_cd"] == 'Z944']

,diag_cd,diag_desc,diag_fst3_cd,diag_fst3_desc,diag_fst4_cd,diag_fst4_desc,icd_ver_cd
56161,Z944,UNKNOWN DIAGNOSIS,Z94,UNKNOWN DIAGNOSIS,Z944,UNKNOWN DIAGNOSIS,9
138443,Z944,LIVER TRANSPLANT STATUS,Z94,TRANSPLANTED ORGAN & TISSUE STATUS,Z944,LIVER TRANSPLANT STATUS,10


In [7]:
lu.icd_ver_cd.value_counts()

10            159478
9             157300
ICD_VER_CD         1
Name: icd_ver_cd, dtype: int64

In [8]:
my_row = lu[lu.icd_ver_cd=="ICD_VER_CD"].index

In [9]:
lu = lu.drop(my_row)

In [10]:
lu['icd_ver_cd']=lu['icd_ver_cd'].astype(int)

## Diagnosis 

In [11]:
f = "final_diag_all"
path = '/ctao_shared_data/Liver_optum/outputs/'
diagnosis = pd.read_csv(path+f+'.csv', delimiter = '\t', low_memory = False)

In [12]:
diagnosis

,patid,fst_dt,icd_flag,diag
0,560499200082764,2017-09-07,10,Z944
1,560499200082764,2017-12-21,10,Z4823
2,560499200082764,2017-12-21,10,Z944
3,560499200082764,2017-12-21,10,E7223
4,560499200082764,2017-12-21,10,Z48298
...,...,...,...,...
16016911,560499899999890,2015-02-11,9,78079
16016912,560499899999890,2015-02-11,9,78605
16016913,560499899999890,2015-02-11,9,3569
16016914,560499899999890,2015-02-11,9,V5869


In [13]:
diagnosis.icd_flag.value_counts()

10    8260060
9     7756856
Name: icd_flag, dtype: int64

## Identify ICD-9 and ICD-10

In [14]:
diagnosis['modified_diag'] = np.where((diagnosis['icd_flag']==9), 'D_9_' + diagnosis['diag'].replace('.', ''), 'D_10_' + diagnosis['diag'].replace('.', ''))

In [15]:
diagnosis

,patid,fst_dt,icd_flag,diag,modified_diag
0,560499200082764,2017-09-07,10,Z944,D_10_Z944
1,560499200082764,2017-12-21,10,Z4823,D_10_Z4823
2,560499200082764,2017-12-21,10,Z944,D_10_Z944
3,560499200082764,2017-12-21,10,E7223,D_10_E7223
4,560499200082764,2017-12-21,10,Z48298,D_10_Z48298
...,...,...,...,...,...
16016911,560499899999890,2015-02-11,9,78079,D_9_78079
16016912,560499899999890,2015-02-11,9,78605,D_9_78605
16016913,560499899999890,2015-02-11,9,3569,D_9_3569
16016914,560499899999890,2015-02-11,9,V5869,D_9_V5869


In [164]:
#diagnosis[(diagnosis.diag == '04100') & (diagnosis.icd_flag == 9)]

In [16]:
col = ['icd_flag', 'diag', 'modified_diag']
diagnosis= diagnosis[col]
diagnosis

,icd_flag,diag,modified_diag
0,10,Z944,D_10_Z944
1,10,Z4823,D_10_Z4823
2,10,Z944,D_10_Z944
3,10,E7223,D_10_E7223
4,10,Z48298,D_10_Z48298
...,...,...,...
16016911,9,78079,D_9_78079
16016912,9,78605,D_9_78605
16016913,9,3569,D_9_3569
16016914,9,V5869,D_9_V5869


In [17]:
diag_filtered = diagnosis.drop_duplicates()
dups = diag_filtered.loc[diag_filtered.diag.duplicated(), :]
dup_list = dups['diag'].tolist()

In [18]:
len(dup_list)

368

In [19]:
dups

,icd_flag,diag,modified_diag
26587,9,E8801,D_9_E8801
122739,9,L851,D_9_L851
122743,9,M79673,D_9_M79673
122744,9,L609,D_9_L609
134832,9,V9999,D_9_V9999
...,...,...,...
15805180,9,S81802D,D_9_S81802D
15805183,9,T66XXXA,D_9_T66XXXA
15857290,10,27652,D_10_27652
15857292,10,5119,D_10_5119


In [23]:
diag_filtered [diag_filtered.diag =="5119"]

,icd_flag,diag,modified_diag
8014,9,5119,D_9_5119
15857292,10,5119,D_10_5119


In [24]:
diag_filtered.modified_diag.nunique()

33214

In [25]:
diag_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33214 entries, 0 to 16011431
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   icd_flag       33214 non-null  int64 
 1   diag           33214 non-null  object
 2   modified_diag  33214 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [171]:
#merged = diag_filtered.merge(lu, left_on='diag', right_on='diag_cd')

In [26]:
merged = diag_filtered.merge(lu, left_on=['diag','icd_flag'], right_on=['diag_cd', 'icd_ver_cd'])

In [29]:
merged.head(10)

,icd_flag,diag,modified_diag,diag_cd,diag_desc,diag_fst3_cd,diag_fst3_desc,diag_fst4_cd,diag_fst4_desc,icd_ver_cd
0,10,Z944,D_10_Z944,Z944,LIVER TRANSPLANT STATUS,Z94,TRANSPLANTED ORGAN & TISSUE STATUS,Z944,LIVER TRANSPLANT STATUS,10
1,10,Z4823,D_10_Z4823,Z4823,ENC AFTERCARE FLW LIVER TRANSPLANT,Z48,ENC OTHER POSTPROCEDURAL AFTERCARE,Z482,ENC AFTERCARE FLW ORGAN TRANSPLANT,10
2,10,E7223,D_10_E7223,E7223,CITRULLINEMIA,E72,OTH DISORDERS AMINO-ACID METABOLISM,E722,DISORDERS OF UREA CYCLE METABOLISM,10
3,10,Z48298,D_10_Z48298,Z48298,ENC AFTERCARE FLW OTH ORGN TRANSPL,Z48,ENC OTHER POSTPROCEDURAL AFTERCARE,Z482,ENC AFTERCARE FLW ORGAN TRANSPLANT,10
4,10,Z23,D_10_Z23,Z23,UNKNOWN DIAGNOSIS,NaN,UNKNOWN DIAGNOSIS,NaN,NaN,10
5,10,Z23,D_10_Z23,Z23,UNKNOWN DIAGNOSIS,Z,UNKNOWN DIAGNOSIS,NaN,NaN,10
6,10,Z23,D_10_Z23,Z23,ENCOUNTER FOR IMMUNIZATION,Z23,ENCOUNTER FOR IMMUNIZATION,NaN,NaN,10
7,10,Z23,D_10_Z23,Z23,UNKNOWN DIAGNOSIS,Z2,UNKNOWN DIAGNOSIS,NaN,NaN,10
8,10,L570,D_10_L570,L570,ACTINIC KERATOSIS,L57,SKIN CHANGES CHRN EXPS NONIONIZ RAD,L570,ACTINIC KERATOSIS,10
9,10,Z08,D_10_Z08,Z08,ENC F/U EX AFTR CMPL TX MALIG NEOPL,Z08,ENC F/U EX AFTR CMPL TX MALIG NEOPL,NaN,NaN,10


In [30]:
merged = merged.loc[merged.diag_desc!='UNKNOWN DIAGNOSIS']
merged

,icd_flag,diag,modified_diag,diag_cd,diag_desc,diag_fst3_cd,diag_fst3_desc,diag_fst4_cd,diag_fst4_desc,icd_ver_cd
0,10,Z944,D_10_Z944,Z944,LIVER TRANSPLANT STATUS,Z94,TRANSPLANTED ORGAN & TISSUE STATUS,Z944,LIVER TRANSPLANT STATUS,10
1,10,Z4823,D_10_Z4823,Z4823,ENC AFTERCARE FLW LIVER TRANSPLANT,Z48,ENC OTHER POSTPROCEDURAL AFTERCARE,Z482,ENC AFTERCARE FLW ORGAN TRANSPLANT,10
2,10,E7223,D_10_E7223,E7223,CITRULLINEMIA,E72,OTH DISORDERS AMINO-ACID METABOLISM,E722,DISORDERS OF UREA CYCLE METABOLISM,10
3,10,Z48298,D_10_Z48298,Z48298,ENC AFTERCARE FLW OTH ORGN TRANSPL,Z48,ENC OTHER POSTPROCEDURAL AFTERCARE,Z482,ENC AFTERCARE FLW ORGAN TRANSPLANT,10
6,10,Z23,D_10_Z23,Z23,ENCOUNTER FOR IMMUNIZATION,Z23,ENCOUNTER FOR IMMUNIZATION,NaN,NaN,10
...,...,...,...,...,...,...,...,...,...,...
33607,9,3720,D_9_3720,3720,ACUTE CONJUNCTIVITIS,372,DISORDERS OF CONJUNCTIVA,3720,ACUTE CONJUNCTIVITIS,9
33608,10,T17298A,D_10_T17298A,T17298A,OTH FOREIGN OBJ PHAR OTH INJ INIT,T17,FOREIGN BODY IN RESPIRATORY TRACT,T172,FOREIGN BODY IN PHARYNX,10
33609,10,T84296S,D_10_T84296S,T84296S,OTH MECH COMP IF DEVC VERTEBRAE SEQ,T84,COMP INTRL ORTHO PROS DEV IMPL GFT,T842,MECH COMP INTERNAL FIX OTH BONES,10
33610,9,29100,D_9_29100,29100,ALCHL WTHDRWL DELIRIUM,291,ALCOHOLIC PSYCHOSES,2910,DELIRIUM TREMENS,9


In [28]:
merged[merged['diag'] == "E8801"]

,icd_flag,diag,modified_diag,diag_cd,diag_desc,diag_fst3_cd,diag_fst3_desc,diag_fst4_cd,diag_fst4_desc,icd_ver_cd
209,10,E8801,D_10_E8801,E8801,ALPHA-1-ANTITRYPSIN DEFICIENCY,E88,OTHER & UNS METABOLIC DISORDERS,E880,DISORDERS PLASMA-PROTEIN METAB NEC,10
2490,9,E8801,D_9_E8801,E8801,ACCIDENTAL FALL/FROM SIDEWALK CURB,E88,FALL,E880,FALL ON STAIRS OR STEPS,9


In [31]:
col = [ 'diag', 'modified_diag', 'diag_desc']
merged = merged[col]
diag_final_desc = merged.drop_duplicates()
diag_final_desc

,diag,modified_diag,diag_desc
0,Z944,D_10_Z944,LIVER TRANSPLANT STATUS
1,Z4823,D_10_Z4823,ENC AFTERCARE FLW LIVER TRANSPLANT
2,E7223,D_10_E7223,CITRULLINEMIA
3,Z48298,D_10_Z48298,ENC AFTERCARE FLW OTH ORGN TRANSPL
6,Z23,D_10_Z23,ENCOUNTER FOR IMMUNIZATION
...,...,...,...
33607,3720,D_9_3720,ACUTE CONJUNCTIVITIS
33608,T17298A,D_10_T17298A,OTH FOREIGN OBJ PHAR OTH INJ INIT
33609,T84296S,D_10_T84296S,OTH MECH COMP IF DEVC VERTEBRAE SEQ
33610,29100,D_9_29100,ALCHL WTHDRWL DELIRIUM


In [176]:
diag_final_desc.modified_diag.nunique()

32003

In [32]:
diag_final_desc[diag_final_desc.diag=='E8801']

,diag,modified_diag,diag_desc
209,E8801,D_10_E8801,ALPHA-1-ANTITRYPSIN DEFICIENCY
2490,E8801,D_9_E8801,ACCIDENTAL FALL/FROM SIDEWALK CURB


In [33]:
diag_final_desc['cat'] = 'diag'
diag_final_desc= diag_final_desc.drop(columns='diag')
diag_final_desc = diag_final_desc.rename(columns = {'modified_diag': 'DIAGNOSIS_ID', 'diag_desc': 'description'})
col = ['DIAGNOSIS_ID','cat','description']
diag_final_desc = diag_final_desc[col]
diag_final_desc

,DIAGNOSIS_ID,cat,description
0,D_10_Z944,diag,LIVER TRANSPLANT STATUS
1,D_10_Z4823,diag,ENC AFTERCARE FLW LIVER TRANSPLANT
2,D_10_E7223,diag,CITRULLINEMIA
3,D_10_Z48298,diag,ENC AFTERCARE FLW OTH ORGN TRANSPL
6,D_10_Z23,diag,ENCOUNTER FOR IMMUNIZATION
...,...,...,...
33607,D_9_3720,diag,ACUTE CONJUNCTIVITIS
33608,D_10_T17298A,diag,OTH FOREIGN OBJ PHAR OTH INJ INIT
33609,D_10_T84296S,diag,OTH MECH COMP IF DEVC VERTEBRAE SEQ
33610,D_9_29100,diag,ALCHL WTHDRWL DELIRIUM


In [184]:
#desc1 = pd.read_csv('desc1.csv')

In [ ]:
#desc1

In [203]:
#desc1[~desc1.isin(diag_final_desc)].dropna()
# common = desc1.merge(diag_final_desc,on=['DIAGNOSIS_ID','cat','description'])
# print(common)
# desc1[(~desc1.DIAGNOSIS_ID.isin(common.DIAGNOSIS_ID))&(~desc1.cat.isin(common.cat))&(~desc1.description.isin(common.description))]

# df = pd.concat([desc1,diag_final_desc])
# df.drop_duplicates(keep=False)

,DIAGNOSIS_ID,cat,description
139,D_10_E8342,diag,OTH FALL WATER TRANSPORT-CREW
146,D_10_E872,diag,FAIL STERILE PRECAUTION DURING PROC
159,D_10_E871,diag,FOREIGN OBJECT LT BODY DURING PROC
165,D_10_E861,diag,ACC POISN CLEAN PAINT VARNISH
205,D_10_E8801,diag,ACCIDENTAL FALL/FROM SIDEWALK CURB
...,...,...,...
31992,D_9_75245,diag,VAGINAL AGENESIS
31997,D_9_87339,diag,OPEN WOUND NOSE MULTIPLE SITES COMP
31998,D_9_3720,diag,ACUTE CONJUNCTIVITIS
32001,D_9_29100,diag,ALCHL WTHDRWL DELIRIUM


In [34]:
diag_final_desc[diag_final_desc.DIAGNOSIS_ID=='D_10_I8381']

,DIAGNOSIS_ID,cat,description
33026,D_10_I8381,diag,VARICOSE VEINS LOWER EXTREM W/PAIN


In [195]:
#desc1[desc1.DIAGNOSIS_ID=='D_10_I8381']

,DIAGNOSIS_ID,cat,description
32005,D_10_I8381,diag,VARICOSE VEINS LOWER EXTREM W/PAIN


In [179]:
#print i

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(i)? (1927283887.py, line 1)

In [ ]:
#dups2 = diag_final_desc.loc[diag_final_desc.modified_diag	.duplicated(keep=False), :]
#dups2
#dups2.to_csv('dups2.csv')

In [ ]:
#diag_final_desc['cat'] = 'diag'
#diag_final_desc= diag_final_desc.drop(columns='diag')
#diag_final_desc = diag_final_desc.rename(columns = {'modified_diag': 'DIAGNOSIS_ID', 'diag_desc': 'description'})
#col = ['DIAGNOSIS_ID','cat','description']
#diag_final_desc = diag_final_desc[col]
#diag_final_desc

# Procedures

In [ ]:
f ='final_proc_all'
path = '/ctao_shared_data/Liver_optum/outputs/'
proc = pd.read_csv(path+f+'.csv', delimiter = '\t', low_memory = False)
proc

In [ ]:
mod_proc_list = []
for proc_code in proc.proc:
    new_code = 'P_' + 'I_' + proc_code.replace('.', '')
    mod_proc_list.append(new_code)
proc['mod_proc_code'] = mod_proc_list
proc

In [ ]:
col = ['icd_flag', 'proc', 'mod_proc_code']
proc_filtered = proc[col].drop_duplicates()
proc_filtered 

In [ ]:
# import the whole lookup table to check merging
import pandas as pd
raw = pd.read_csv('/ctao_shared_data/Optum_lookup/lu_procedure.csv', delimiter='\t', low_memory=False)
raw

In [ ]:
#lu = raw[['diag_cd','diag_desc','diag_fst3_cd','diag_fst3_desc', 'diag_fst4_cd','diag_fst4_desc','icd_ver_cd']]
lu = raw.loc[raw.proc_desc!='UNKNOWN PROCEDURE']
lu = lu.drop_duplicates().reset_index(drop=True)
lu.isnull().sum()

In [ ]:
lu

In [ ]:
merged2 = proc_filtered.merge(lu, left_on='proc', right_on='proc_cd')

In [ ]:
merged2

In [ ]:
col = [ 'proc', 'mod_proc_code', 'proc_desc']
merged2 = merged2[col]
proc_final_desc = merged2.drop_duplicates()
proc_final_desc

In [ ]:
#'DIAGNOSIS_ID','cat','description'
proc_final_desc['cat'] = 'proc'
proc_final_desc= proc_final_desc.drop(columns='proc')
proc_final_desc = proc_final_desc.rename(columns = {'mod_proc_code': 'DIAGNOSIS_ID', 'proc_desc': 'description'})
col = ['DIAGNOSIS_ID','cat','description']
proc_final_desc = proc_final_desc[col]
proc_final_desc

## Medications

In [ ]:
f ='mod_drug'
ot_path = '../Data_V2/'
med = pd.read_csv(ot_path+f+'.csv', low_memory = False)
med

In [ ]:
med_final_desc = med [['patid','mod_gen_drug']]
med_final_desc['cat'] = 'drug'
med_final_desc['description'] = med_final_desc['mod_gen_drug']
#med_final_desc = med['mod_gen_drug']
med_final_desc= med_final_desc.drop(columns='patid')
med_final_desc = med_final_desc.drop_duplicates()
med_final_desc 

In [ ]:
med_final_desc = med_final_desc.rename(columns = {'mod_gen_drug': 'DIAGNOSIS_ID'})
col = ['DIAGNOSIS_ID','cat','description']
med_final_desc = med_final_desc[col]
med_final_desc 

## Combining All Descriptions

In [ ]:
desc = pd.concat([diag_final_desc ,proc_final_desc ,med_final_desc ])

In [ ]:
desc

In [ ]:
ot_path = '../Data_V2/'
desc.to_csv(ot_path+'descriptions.csv',index=False)